<img src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="200"/>

# Getting Started with the Arize Platform - Customer Lifetime Value in Telecommunication Industry

**You are part of a team in a telecommunication company that monitors and maintains a customer lifetime value (LTV) regression model, which predicts the LTV for each customer.** The business objective of this regression model is to accurately predict customer lifetime value in order to improve customer segmentation and profiling to customize offers and target customers based on their potential value and recognize best customers.

You understand that flaws in your model performance will have a huge negative impact on your company and with your LTV model in production you don't have any effective tool at your disposal to monitor the performance of your models, identify any issues and troubleshoot costly model degradations. Therefore, you turn to Arize to help you understand what went wrong in your model and how you can improve it. 


**In this walkthrough, we are going to investigate your production LTV model. We will validate degradation in model performance, take a deep dive to investigate the root causes of those inaccurate predictions, and set up proactive monitors to mitigate the impact of future degradations.**


You will learn how to:

1.   Get training and production data into the Arize platform
2.   Setup performance dashboards and monitors to look at prediction performance
3.   Understand where the model is underperforming
4.   Discover the root cause of issues
5. Set up pro-active monitoring to mitigate the impact of such degradations in the future


The production data contains 1 month of data where 2 main issues exist. You will work on identifying these issues over the course of this exercise.

1.   A data source has introduced changes in the distribution of particular features
2.   The model is inacurate during some time period due to particular features 



# Step 0. Setup and Getting the Data

The first step is to load our preexisting dataset which includes training and production environments for our LTV model. Using a preexisting dataset illustrates how simple it is to get started with the Arize platform.


## Install Dependencies and Import Libraries 📚

In [ ]:
!pip install -q arize

import datetime
import uuid
from datetime import timedelta

import pandas as pd
from arize.pandas.logger import Client, Schema
from arize.utils.types import Environments, ModelTypes

## **🌐 Download the Data**
In this walkthrough, we’ll be sending real historical data. Note, that while feature names and values are made explicit in this dataset, you can achieve the same level of ML Observability using obfuscated features. 

| Feature | Type | Description |||
|:-|:-|:-|---|---|
| `City`| `str`| city in California where the customer resides |||
| `Gender`| `str`| customer's gender |||
| `Partner`| `str`| flag indicating if the customer has a partner |||
| `Dependents`| `str`| flag indicating if the customer has dependents |||
| `Phone Service`| `str `| flag indicating if the customer has phone service |||
| `Internet Service`| `str`| flag indicating if the customer has internet service |||
| `Streaming TV`| `str`| flag indicating if the customer streams TV |||
| `Streaming Movies`| `str`| flag indicating if the customer streams movies |||
| `Churn Value`| `int (0 or 1)`| flag indicating if the customer churned |||

## Inspect the Data 

The data represents a regression model trained to predict LTV for a customer. The dataset contains one month of data and the performance will be evaluated by comparing:

*   **`Predicted LTV`**: Predicted value of LTV for each customer
*   **`Actual LTV`**: Actual value of LTV for each customer




In [ ]:
# Preparing dataset for this tutorial
train_df = pd.read_csv(
    "https://storage.googleapis.com/arize-assets/fixtures/LTV%20Use-Case/LTV_train.csv"
)
test_df = pd.read_csv(
    "https://storage.googleapis.com/arize-assets/fixtures/LTV%20Use-Case/LTV_test.csv"
)


print("✅ Dependencies installed and data successfully downloaded!")

## Inspect and Prepare the Data 

In [ ]:
# Preparing Training Data
train_df["prediction_id"] = [str(uuid.uuid4()) for _ in range(len(train_df))]
train_df = train_df.drop(columns=["Unnamed: 0"])
train_df

In [ ]:
# Preparing production data
def prod_ID_time(df, start, end):
    max_d = df["day"].max()
    out_df = pd.DataFrame()
    dts = pd.date_range(start, end).to_pydatetime().tolist()
    for dt in dts:
        day_df = df.loc[df["day"] == (dt.day % max_d)].copy()
        day_df["prediction_ts"] = int(dt.strftime("%s"))
        out_df = pd.concat([out_df, day_df], ignore_index=True)
    out_df["prediction_id"] = [str(uuid.uuid4()) for _ in range(out_df.shape[0])]
    return out_df.drop(columns="day")


today = datetime.date.today()
END_DATE = (today).strftime("%Y-%m-%d")
START_DATE = (today - timedelta(31)).strftime("%Y-%m-%d")

test_df = prod_ID_time(test_df, START_DATE, END_DATE)
test_df = test_df.drop(columns=["Unnamed: 0"])

test_df

# Step 1. Sending Data into Arize 💫

Now that we have our dataset imported, we are ready to integrate into Arize. We do this by logging (sending) important data we want to analyze to the platform. There, the data will be easily visualized and troubleshooting workflows will help us find the source of our problem.

For our model, we are going to log:
*   feature data
*   predictions
*   actuals

## Import and Setup Arize Client

The first step is to setup our Arize client. After that we will log the data.

First, copy the Arize `API_KEY` and `SPACE_KEY` from your admin page shown below! Copy those over to the set-up section. We will also be setting up some metadata to use across all logging.




<img src="https://storage.googleapis.com/arize-assets/fixtures/copy-keys.png" width="700">

In [ ]:
SPACE_KEY = "SPACE_KEY"
API_KEY = "API_KEY"
arize_client = Client(space_key=SPACE_KEY, api_key=API_KEY)

# Saving model metadata for passing in later
model_id = "customer-lifetime-value-demo-model"  # This is the model name that will show up in Arize
model_version = "v1.0"  # Version of model - can be any string

if SPACE_KEY == "SPACE_KEY" or API_KEY == "API_KEY":
    raise ValueError("❌ NEED TO CHANGE SPACE AND/OR API_KEY")
else:
    print("✅ Arize setup complete!")

## Log Training & Production Data to Arize 

Now that our Arize client is setup, let's go ahead and log all of our data to the platform. For more details on how **`arize.pandas.logger`** works, visit out documentations page below.

[![Buttons_OpenOrange.png](https://storage.googleapis.com/arize-assets/fixtures/Buttons_OpenOrange.png)](https://docs.arize.com/arize/sdks-and-integrations/python-sdk/arize.pandas)

Key parameters:

*   **prediction_label_column_name**: tells Arize which column contains the predictions
*   **actual_label_column_name**: tells Arize which column contains the actual results from field data


We will use [ModelTypes.NUMERIC](https://docs.arize.com/arize/concepts-and-terminology/model-types) to perform this analysis.




### Log the training data for your model to Arize!

In [ ]:
# Define a Schema() for Arize to pick up the data from the correct column for logging

train_schema = Schema(
    prediction_id_column_name="prediction_id",
    prediction_label_column_name="Predicted LTV",
    actual_label_column_name="Actual LTV",
    feature_column_names=train_df.columns.drop(
        ["prediction_id", "Predicted LTV", "Actual LTV"]
    ),
)

train_res = arize_client.log(
    dataframe=train_df,
    model_id=model_id,
    model_version=model_version,
    model_type=ModelTypes.NUMERIC,
    environment=Environments.TRAINING,
    schema=train_schema,
)
if train_res.status_code != 200:
    print(f"future failed with response code {train_res.status_code}, {train_res.text}")
else:
    print(f"future completed with response code {train_res.status_code}")

### Log the production data
Note: We will be sending our test data to emulate sending production data.

In [ ]:
# Logging production
all_cols = test_df.columns
feature_cols = all_cols.drop(
    ["prediction_id", "prediction_ts", "Predicted LTV", "Actual LTV"]
)

test_schema = Schema(
    prediction_id_column_name="prediction_id",
    timestamp_column_name="prediction_ts",
    prediction_label_column_name="Predicted LTV",
    actual_label_column_name="Actual LTV",
    feature_column_names=feature_cols,
)

test_res = arize_client.log(
    dataframe=test_df,
    model_id=model_id,
    model_version=model_version,
    model_type=ModelTypes.NUMERIC,
    environment=Environments.PRODUCTION,
    schema=test_schema,
)
if test_res.status_code != 200:
    print(f"future failed with response code {test_res.status_code}, {test_res.text}")
else:
    print(f"future completed with response code {test_res.status_code}")

# Step 2. Confirm Data in Arize ✅

Note that the Arize performs takes about 10 minutes to index the data. While the model should appear immediately, the data will not show up until the indexing is complete. Feel free to go grab a cup of coffee as Arize works its magic! ☕🔮

**⚠️ DON'T SKIP:**
In order to move on to the next step, make sure your actuals and training/production sets are loaded into the platform. To check:
1. Navigate to models from the left bar, locate and click on model **LTV-use-case-tutorial**
2. On the **Overview Tab**, make sure you can see Predictions and Actuals under the **Model Health** section. Once production actuals have been fully recorded on Arize, the row title will change from **0 Actuals** to **Actuals** with summary statistics such as cardinality listed in the tables.
3. Verify the list of **Features** below **Actuals**.

![image.png](https://storage.googleapis.com/arize-assets/fixtures/LTV%20Use-Case/newmodel.gif)

# Step 3. Set up Model Baseline & Managed Monitors

Now that our data has been logged into the [Arize platform](https://app.arize.com/) we can begin our investigation into our poorly performing LTV regression model. 

Arize will guide you through setting up a **Baseline** (reference environment for comparison) and automatically create **Monitors** for your model in just a few clicks —just follow the blue banner at the top of the page titled "Finish setting up your model". 

![image.png](https://storage.googleapis.com/arize-assets/fixtures/Click-Through%20Rate%20Use-Case/images/initial_setup_banner.png)

Arize can automatically configure monitors that are best suited to your data. From the banner at the top of the screen, select the following configurations after clicking the 'Set up Model' button: 

1. Datasets: `Training Version 1.0`
2. Default Metric: `RMSE`, Trigger Alert When: `RMSE is above 80`
3. Turn On Monitoring: Drift ✅, Data Quality ✅, Performance ✅ 

You will now see that the baseline has been set and **Drift**, **Data Quality**, and **Performance** monitors have been created!!! 

![image](https://storage.googleapis.com/arize-assets/fixtures/LTV%20Use-Case/modelsetup.gif)

Arize automatically sets up monitors to ensure our model is flagged if the Performance, Data Quality, or Drift spikes above a certain threshold/before it becomes a major issue. You can see filter monitors by category, edit evaluation windows, thresholds, etc. and create custom monitors by visiting the **Monitors** tab.


# Step 4. Exploring Drift 

Selecting the **Drift** tab we notice that we do not see any noticeable model prediction drift in production compared to our training dataset (baseline).  
It's important to note that while we do not see any prediction drift, we do notice an increase in RMSE between day 22 and day 3.

![image](https://storage.googleapis.com/arize-assets/fixtures/LTV%20Use-Case/Screen%20Shot%202021-10-18%20at%2011.48.45%20PM.png)



# Step 5. Analyzing feature drift
During intial model setup, Arize automatically created a set drift monitors and dashboards for each feature available in the dataset. Drift is represented as the Population Stability Index (PSI) over an given period. We will use these graphs to monitor overall trends. From the model overview page, select the **Internet Service** feature and you should notice the change in Internet Service distribution. Use the PSI graph to select a period of interest. Notice that the default threshold set by Arize was crossed for the same period that the RMSE spiked.

![image.png](https://storage.googleapis.com/arize-assets/fixtures/LTV%20Use-Case/Screen%20Shot%202021-10-18%20at%2011.57.16%20PM.png)




# Step 6. Setting up a proactive custom monitor 


![image](https://storage.googleapis.com/arize-assets/fixtures/LTV%20Use-Case/Screen%20Shot%202021-10-19%20at%2011.37.06%20AM.png)

# Step 7. Analyzing Root Cause 

Arize facilitates troubleshooting which features and more specifically slices (feature/value combinations) could be the culprit of our model's performance degradation. Arize provides a number of flexible dashboard templates for users to get started in creating custom layouts for surfacing model insights. Let's see how in just a few clicks we can quickly identify which features could be causing our model to perform so badly. Navigate to the **Templates** section on the left sidebar and scroll down to click on the **Feature Performance Heatmap**. From there select your model, select the features you care to investigate, metric `RMSE`. 

![image](https://storage.googleapis.com/arize-assets/fixtures/LTV%20Use-Case/heatmap.gif)

Arize automatically discovers and surfaces model performance issues across all features and various feature/value combinations. Visual indicators facilitate drill down analysis of the most problematic slices affecting your overall model performance. 

Some of the immediate insights surfaced from our **Feature Performance Heatmap** are detailed below. 
1. Worst performing `Internet Service` is `Fiber Optic`
2. Worst performing `Streaming TV` is `Yes`
3. Worst performing `Streaming Movies` is `Yes`
4. Worst performing `City` includes `Moreno Valley`, `Roseville`, and `Novato.`

![image](https://storage.googleapis.com/arize-assets/fixtures/LTV%20Use-Case/Screen%20Shot%202021-10-18%20at%2010.00.49%20PM.png)
![image](https://storage.googleapis.com/arize-assets/fixtures/LTV%20Use-Case/Screen%20Shot%202021-10-18%20at%209.59.58%20PM.png)
![image](https://storage.googleapis.com/arize-assets/fixtures/LTV%20Use-Case/Screen%20Shot%202021-10-18%20at%2010.00.16%20PM.png)



We can further dive into a performance analysis of our model by filtering on various low performing feature/value slices. This will help determine the data needed to upsample and retrain our model to improve performance on these segments. For example, by adding a filter in our **Feature Performance Heatmap** we can narrow down where the model degradation is originating from and through which features it is most prominent. 

![image](https://storage.googleapis.com/arize-assets/fixtures/LTV%20Use-Case/Screen%20Shot%202021-10-19%20at%2011.22.55%20AM.png)


# Step 8. Feature Performance Insights --> Actions 

Using the insights gathered from our **Feature Performance Heatmap**, we can take action to use our findings to improve our model. As noted previously, the slice (feature/value combination) `Streaming TV: Yes`, along with `Internet Service` and `Streaming Movies`, was one of the worst performing segments for our model. Navigate back to the model **Overview** page and click into the `Streaming TV` feature under the **Features** section. 

Upon first glance, we see that `Streaming TV` is starting to drift around day 21 where the model experienced more data from the `Yes` variable, which was also accompanied by an increase in RMSE during the same time frame.

![image](https://storage.googleapis.com/arize-assets/fixtures/LTV%20Use-Case/Screen%20Shot%202021-10-19%20at%2012.02.18%20AM.png)


Similarly, by navigating to `City` we notice significant drift across training and production distributions across various regions like `Novato` and `Roseville`. Furthermore, we notice that our training dataset (Baseline distribution) had little to no values for these cities. 

![image](https://storage.googleapis.com/arize-assets/fixtures/LTV%20Use-Case/Screen%20Shot%202021-10-19%20at%2012.10.23%20AM.png)

This troubleshooting flow has lead us to understand that retraining the model on these slices of feature/value combinations (namely `City`:`Novato, Roseville`, `Streaming TV`: `Yes` and  `Internet Service`: `Yes`) could significantly improve our model's RMSE performance.

# Step 9. Model Performance Overview

We can get a quick performance overview by clicking on the performace tab under your specific model. 

![image](https://storage.googleapis.com/arize-assets/fixtures/LTV%20Use-Case/Screen%20Shot%202021-10-19%20at%2011.42.50%20AM.png)


As we continue to check in and improve our model's performance, we want to be able to efficiently view all our important model metrics in a single pane. In the same way we set up a **Feature Performance Heatmap** we will create a **Model Performance Dashboard** to view our model's most important metrics in a single configurable layout. 

Navigate to the **Templates** section on the left sidebar and scroll down to click on the **Regression Model**. From there select your model, features you care to investigate. 

![image](https://storage.googleapis.com/arize-assets/fixtures/LTV%20Use-Case/regmodel.gif)

In addition to the default widgets Arize sets up for your dashboard, you can configure custom metrics your team cares about. In only a few clicks I added a few widgets to give me a single glance view of my model's **RMSE**, **MAPE**, and **MAE** as standalone statistics widgets. To visualize these metrics over time I also created a timeseries widget and overlayed three plots showcase the fluctuation of my metrics over time. 

![image](https://storage.googleapis.com/arize-assets/fixtures/LTV%20Use-Case/Screen%20Shot%202021-10-19%20at%2011.17.11%20AM.png)


# 📚 Conclusion 

Customer lifetime value is an important metric to consider in every business. Use Arize to identify segments where your LTV model is underperforming, troubleshoot root cause analysis, and proactively monitor for future degradations.

# About Arize
Arize is an end-to-end ML observability and model monitoring platform. The platform is designed to help ML engineers and data science practitioners surface and fix issues with ML models in production faster with:
- Automated ML monitoring and model monitoring
- Workflows to troubleshoot model performance
- Real-time visualizations for model performance monitoring, data quality monitoring, and drift monitoring
- Model prediction cohort analysis
- Pre-deployment model validation
- Integrated model explainability

### Website
Visit Us At: https://arize.com/model-monitoring/

### Additional Resources
- [What is ML observability?](https://arize.com/what-is-ml-observability/)
- [Playbook to model monitoring in production](https://arize.com/the-playbook-to-monitor-your-models-performance-in-production/)
- [Using statistical distance metrics for ML monitoring and observability](https://arize.com/using-statistical-distance-metrics-for-machine-learning-observability/)
- [ML infrastructure tools for data preparation](https://arize.com/ml-infrastructure-tools-for-data-preparation/)
- [ML infrastructure tools for model building](https://arize.com/ml-infrastructure-tools-for-model-building/)
- [ML infrastructure tools for production](https://arize.com/ml-infrastructure-tools-for-production-part-1/)
- [ML infrastructure tools for model deployment and model serving](https://arize.com/ml-infrastructure-tools-for-production-part-2-model-deployment-and-serving/)
- [ML infrastructure tools for ML monitoring and observability](https://arize.com/ml-infrastructure-tools-ml-observability/)

Visit the [Arize Blog](https://arize.com/blog) and [Resource Center](https://arize.com/resource-hub/) for more resources on ML observability and model monitoring.
